# Collect Demonstration from Keyboard

Collect demonstration data for the given environment.
The task is to pick a mug and place it on the plate. The environment recognizes the success if the mug is on the plate, gthe ripper opened, and the end-effector positioned above the mug.

<img src="./media/teleop_v2.gif" width="480" height="360">

Use WASD for the xy plane, RF for the z-axis, QE for tilt, and ARROWs for the rest of rthe otations. 

SPACEBAR will change your gripper's state, and Z key will reset your environment with discarding the current episode data.

For overlayed images, 
- Top Right: Agent View 
- Bottom Right: Egocentric View
- Top Left: Left Side View
- Bottom Left: Top View

In [1]:
# Cell 1 - 设置环境变量（必须第一个运行）
import os

# 1. 设置DISPLAY
os.environ['DISPLAY'] = ':0'
os.environ['XAUTHORITY'] = os.path.expanduser('~/.Xauthority')
print(f"✓ DISPLAY设置为: {os.environ['DISPLAY']}")

# 2. 强制使用GPU渲染（关键！）
os.environ['MUJOCO_GL'] = 'egl'  # EGL后端GPU加速
print(f"✓ MUJOCO_GL: egl (GPU硬件加速)")

# 3. NVIDIA GPU优化
os.environ['__GL_SYNC_TO_VBLANK'] = '0'  # 关闭垂直同步
os.environ['__GL_YIELD'] = 'NOTHING'      # 减少CPU等待
print("✓ NVIDIA GPU优化已启用")

# 4. OpenGL性能优化
os.environ['__GL_FSAA_MODE'] = '0'        # 关闭抗锯齿
os.environ['__GL_LOG_MAX_ANISO'] = '0'    # 关闭各向异性过滤
print("✓ OpenGL性能优化已启用")

✓ DISPLAY设置为: :0
✓ MUJOCO_GL: egl (GPU硬件加速)
✓ NVIDIA GPU优化已启用
✓ OpenGL性能优化已启用


In [2]:
import sys
import random
import numpy as np
import os
from PIL import Image
from mujoco_env.y_env2 import SimpleEnv2
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

/home/wzy/envs/lerobot-mujoco/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# If you want to randomize the object positions, set this to None
# If you fix the seed, the object positions will be the same every time
SEED = 0 
# SEED = None <- Uncomment this line to randomize the object positions

REPO_NAME = 'omy_pnp_language'
NUM_DEMO = 20 # Number of demonstrations to collect
ROOT = "./251104-myself_data_language" # The root directory to save the demonstrations

In [4]:
#xml_path = './asset/example_scene_y2.xml'
xml_path = '/home/wzy/lerobot-mujoco/01-Parol6/parol6.xml'
# Define the environment
PnPEnv = SimpleEnv2(xml_path, seed = SEED, state_type = 'joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[6] n_qvel:[6] n_qacc:[6] n_ctrl:[6]
 integrator:[EULER]

n_body:[8]
 [0/8] [world] mass:[0.00]kg
 [1/8] [base_link] mass:[0.81]kg
 [2/8] [L1] mass:[0.64]kg
 [3/8] [L2] mass:[0.51]kg
 [4/8] [L3] mass:[0.55]kg
 [5/8] [L4] mass:[0.39]kg
 [6/8] [L5] mass:[0.19]kg
 [7/8] [L6] mass:[0.07]kg
body_total_mass:[3.17]kg

n_geom:[8]
geom_names:['floor', None, None, None, None, None, None, None]

n_mesh:[7]
mesh_names:['base_link', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6']

n_joint:[6]
 [0/6] [L1] axis:[0. 0. 1.]
 [1/6] [L2] axis:[0. 0. 1.]
 [2/6] [L3] axis:[ 0.  0. -1.]
 [3/6] [L4] axis:[ 0.  0. -1.]
 [4/6] [L5] axis:[ 0.  0. -1.]
 [5/6] [L6] axis:[ 0.  0. -1.]

n_dof:[6] (=number of rows of Jacobian)
 [0/6] [None] attached joint:[L1] body:[L1]
 [1/6] [None] attached joint:[L2] body:[L2]
 [2/6] [None] attached joint:[L3] body:[L3]
 [3/6] [None] attached joint:[L4] body:[L4]
 [4/6] 

KeyError: "Invalid name 'joint1'. Valid names: ['L1', 'L2', 'L3', 'L4', 'L5', 'L6']"

## Define Dataset Fatures and Create your dataset!
The dataset is contained as follows:
```
fps = 20,
features={
    "observation.image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channels"],
    },
    "observation.wrist_image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channel"],
    },
    "observation.state": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["state"], # x, y, z, roll, pitch, yaw
    },
    "action": {
        "dtype": "float32",
        "shape": (7,),
        "names": ["action"], # 6 joint angles and 1 gripper
    },
    "obj_init": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["obj_init"], # just the initial position of the object. Not used in training.
    },
},
```


This will make the dataset on './demo_data' folder, which will look like this,

```
.
├── data
│   ├── chunk-000
│   │   ├── episode_000000.parquet
│   │   └── ...
├── meta
│   ├── episodes.jsonl
│   ├── info.json
│   ├── stats.json
│   └── tasks.jsonl
└── 
```


In [14]:
create_new = True
if os.path.exists(ROOT):
    print(f"Directory {ROOT} already exists.")
    ans = input("Do you want to delete it? (y/n) ")
    if ans == 'y':
        import shutil
        shutil.rmtree(ROOT)
    else:
        create_new = False


if create_new:
    dataset = LeRobotDataset.create(
                repo_id=REPO_NAME,
                root = ROOT, 
                robot_type="omy",
                fps=20, # 20 frames per second
                features={
                    "observation.image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channels"],
                    },
                    "observation.wrist_image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channel"],
                    },
                    "observation.state": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["state"], # x, y, z, roll, pitch, yaw
                    },
                    "action": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["action"], # 6 joint angles and 1 gripper
                    },
                    "obj_init": {
                        "dtype": "float32",
                        "shape": (9,),
                        "names": ["obj_init"], # just the initial position of the object. Not used in training.
                    },
                },
                image_writer_threads=10,
                image_writer_processes=5,
        )
else:
    print("Load from previous dataset")
    dataset = LeRobotDataset(REPO_NAME, root=ROOT)

Directory ./demo_data_language already exists.


Process Process-10:
Process Process-7:
Process Process-8:
Process Process-6:
Process Process-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/

## Keyboard Control
You can teleop your robot with keyboard and collect dataset
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```
Reseting your environment will remove the cache data of the current demonstration and restart collection.

### Now let's teleop our robot and collect data!

**To receive the success signal, you have to release the gripper and move upwards above the mug!**

In [15]:
# 初始化一个7维的动作向量（全为0）
# 这7个维度包括：6个机器人关节角度 + 1个夹爪开合状态
action = np.zeros(7)

# 当前演示回合的编号，从0开始计数
episode_id = 0

# 记录标志位：用于控制何时开始记录数据
# False表示还未开始记录，当机器人开始移动时才设为True
record_flag = False  # Start recording when the robot starts moving

# 主循环：持续运行直到以下任一条件不满足
# 1. 仿真查看器窗口还在运行
# 2. 已收集的演示数量还未达到目标数量NUM_DEMO
while PnPEnv.env.is_viewer_alive() and episode_id < NUM_DEMO:
    
    # 推进环境一步（更新物理仿真）
    PnPEnv.step_env()
    
    # 控制主循环频率为20Hz（每秒执行20次）
    # 这样可以避免执行太快导致数据采集过密
    if PnPEnv.env.loop_every(HZ=20):
        
        # ===== 检查任务完成状态 =====
        done = PnPEnv.check_success()
        if done:  # 如果任务成功完成
            # 保存当前回合的所有数据到数据集
            dataset.save_episode()
            # 重置环境，准备下一个回合
            PnPEnv.reset()
            # 回合计数器加1
            episode_id += 1
        
        # ===== 遥操作控制 =====
        # 通过遥操作设备（如键盘、手柄等）控制机器人
        # action: 末端执行器的增量姿态变化 + 夹爪状态
        # reset: 是否按下了重置键（如'z'键）
        action, reset = PnPEnv.teleop_robot()
        
        # 判断是否应该开始记录数据
        # 条件：还未开始记录 且 有实际动作输入（action不全为0）
        if not record_flag and sum(action) != 0:
            record_flag = True
            print("Start recording")  # 提示开始记录
        
        # 如果用户按下了重置键
        if reset:
            # 重置环境到初始状态
            PnPEnv.reset()
            # 清空当前回合的缓冲数据（丢弃失败的尝试）
            dataset.clear_episode_buffer()
            # 停止记录
            record_flag = False
        
        # ===== 获取观测数据 =====
        # 从两个摄像头获取RGB图像
        # agent_image: 第三人称视角的图像（固定摄像头）
        # wrist_image: 第一人称视角的图像（安装在机械臂腕部的摄像头）
        agent_image, wrist_image = PnPEnv.grab_image()
        
        # ===== 图像预处理 =====
        # 将numpy数组转换为PIL Image对象，方便进行resize操作
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)
        
        # 将图像统一调整为256x256的尺寸
        # 这是为了满足后续神经网络模型的输入要求
        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))
        
        # 将PIL Image转回numpy数组，用于数据存储
        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)
        
        # ===== 执行动作并获取机器人状态 =====
        # 让机器人执行action对应的动作，并返回当前关节角度
        joint_q = PnPEnv.step(action)
        
        # 获取实际的机器人状态（7维）
        # PnPEnv.q包含了机器人的完整状态，取前7个维度
        # 这7个维度是：6个关节的实际角度 + 1个夹爪的实际开合度
        action = PnPEnv.q[:7]
        
        # 转换为float32类型，节省存储空间
        action = action.astype(np.float32)
        
        # ===== 数据记录 =====
        # 只有在record_flag为True时才记录数据
        # 这样可以避免记录机器人静止不动时的无效数据
        if record_flag:
            # 将当前帧的所有数据添加到数据集
            dataset.add_frame({
                "observation.image": agent_image,        # 第三人称视角图像
                "observation.wrist_image": wrist_image,  # 第一人称视角图像
                "observation.state": joint_q[:6],        # 机器人状态（6个关节角度）
                "action": action,                         # 动作（7维：6关节+1夹爪）
                "obj_init": PnPEnv.obj_init_pose,       # 物体初始位姿
                # "task": PnPEnv.instruction,            # 任务描述（已注释）
            }, task=PnPEnv.instruction  # 任务指令（如"pick and place"）
            )
        
        # ===== 可视化渲染 =====
        # 更新仿真环境的可视化界面
        # teleop=True: 表示当前处于遥操作模式
        # idx=episode_id: 显示当前回合编号
        PnPEnv.render(teleop=True, idx=episode_id)

Start recording


Creating parquet from Arrow format: 100%|██████████| 7/7 [00:00<00:00, 99.29ba/s]


DONE INITIALIZATION


Creating parquet from Arrow format: 100%|██████████| 5/5 [00:00<00:00, 72.39ba/s]


DONE INITIALIZATION


...

Creating parquet from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 90.21ba/s]


DONE INITIALIZATION
DONE INITIALIZATION
Start recording


Creating parquet from Arrow format: 100%|██████████| 4/4 [00:00<00:00, 87.30ba/s]


DONE INITIALIZATION


KeyboardInterrupt: 

In [16]:
PnPEnv.env.close_viewer()

In [17]:
# Clean up the images folder
import shutil
shutil.rmtree(dataset.root / 'images')